In [1]:
import os

In [2]:
%pwd


'd:\\Projects\\Chicken_Disease_classification\\research'

In [3]:
os.chdir("/projects/Chicken_Disease_classification")

In [4]:
%pwd

'd:\\projects\\Chicken_Disease_classification'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [44]:
from Chicken_Disease_classification.constants import *
from Chicken_Disease_classification.utils.common import read_yaml, create_directory



In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig: # this is a method of the class ConfigurationManager for preparing the callbacks
        config = self.config.prepare_callbacks
        

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(self.config.artifacts_root),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )

        create_directory([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config


In [46]:
import time 

In [47]:
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [48]:
import os
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile
import time


In [49]:
###TESTING

In [50]:
#----------------------#----------------------#
#----------------------#----------------------#

In [51]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Eager execution enabled:", tf.executing_eagerly())

TensorFlow version: 2.19.0
Eager execution enabled: True


In [54]:
import tensorflow as tf
from pathlib import Path


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the model from the path specified in config
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        # Arguments for the ImageDataGenerator
        datagenerator_kwargs = dict(
            rescale=1./255,  # Normalize image values to [0,1]
            validation_split=0.20  # Split 20% of data for validation
        )

        # Data flow arguments for resizing, batch size, etc.
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Image size excluding channels
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Creating the validation data generator
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",  # Only use the validation subset
            shuffle=False,
            **dataflow_kwargs
        )

        # Train data generator with augmentation if needed
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator  # Use the same generator if no augmentation

        # Creating the training data generator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",  # Only use the training subset
            shuffle=True,  # Shuffle data
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the trained model to the specified path
        model.save(path)

    
    def train(self, callback_list: list):
        # Compute steps per epoch and validation steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        try:
            # Train the model using the train and validation generators
            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                callbacks=callback_list
            )
        except Exception as e:
            print(f"Error during training: {e}")
            raise e

        # Save the trained model to the specified path
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [55]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

[2025-04-04 19:07:31,111] INFO: Successfully read YAML file 'Config\config.yaml'.
[2025-04-04 19:07:31,117] INFO: Successfully read YAML file 'params.yaml'.
[2025-04-04 19:07:31,119] INFO: Created directory at 'artifacts'.
[2025-04-04 19:07:31,123] INFO: Created directory at 'artifacts\prepare_callbacks\tensorboard_log_dir'.
[2025-04-04 19:07:31,125] INFO: Created directory at 'artifacts\training'.
[2025-04-04 19:07:31,552] WARNING: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


d:\Projects\Chicken_Disease_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Error during training: numpy() is only available when eager execution is enabled.


NotImplementedError: numpy() is only available when eager execution is enabled.